In [1]:
import __init__
import qqespm_quadtree_CGA as qq2
import geojson
import shapely
import os
import json
geojson.geometry.DEFAULT_PRECISION = 6

In [2]:
column_names = ['amenity','shop','tourism','landuse','leisure','building']

In [3]:
def pois_to_geojson_features(pois):
    features = []
    for i, row in pois.iterrows():
        geometry = json.loads(shapely.to_geojson(row['geometry']))
        centroid = json.loads(shapely.to_geojson(row['centroid']))
        keywords = []
        for column in column_names:
            if type(row[column]) == str:
                keywords.append(row[column])
        if type(row['name']) == str:
            name = row['name']
        else:
            name = ''
        feature = geojson.Feature(geometry=geometry, 
                                  centroid=centroid,
                    properties={'keywords': keywords,
                                'name': name,
                                'osm_id': row['osm_id']})
        #feature['centroid'] = geojson.geometry.Point(shapely.geometry.shape(feature['geometry']).centroid.coords[0])
        features.append(feature)
        
    features = geojson.FeatureCollection(features)
    print('Total features imported:', len(features.features))
    print('Geometry types found:', set([features['features'][i]['geometry']['type'] for i in range(len(features['features']))]))
    return features

In [4]:
percs = ['20perc', '40perc', '60perc', '80perc', '100perc']
base_dataset_filename = 'london_pois_bbox'

In [5]:
pois = {}
features = {}
for perc in percs:
    pois[perc] = qq2.read_df_csv(data_dir = f'../data/{base_dataset_filename}_{perc}.csv')
    features[perc] = pois_to_geojson_features(pois[perc])
    with open(f'../data/{base_dataset_filename}_{perc}.geojson', 'w') as f:
        geojson.dump(features[perc], f)

Total features imported: 25595
Geometry types found: {'Polygon'}
Total features imported: 51190
Geometry types found: {'Polygon'}
Total features imported: 76785
Geometry types found: {'Polygon'}
Total features imported: 102380
Geometry types found: {'Polygon'}
Total features imported: 127975
Geometry types found: {'Polygon'}
